In [75]:
# Markov chain
# import libraries
import numpy as np
import pandas as pd
import yfinance as yf

# **What is a stochastic process?**
**A stochastic process is a collection of random
variables: $\{X_t,t \in I \},$ Where: $X_t$ is the set of random variables at time $t$, and $I$ is the index set of the process.**


In [76]:
SPY = yf.download("SPY", start="2018-01-01", end="2018-12-31")
data = SPY[['Open', 'High', 'Low', 'Adj Close']].copy()
data.tail()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Adj Close
Date,,,,
2018-12-21,246.740005,249.710007,239.979996,229.649750
2018-12-24,239.039993,240.839996,234.270004,223.581726
2018-12-26,235.970001,246.179993,233.759995,234.878159
2018-12-27,242.570007,248.289993,238.960007,236.681427
2018-12-28,249.580002,251.399994,246.449997,236.376099


In [77]:
# compute the daily returns
data['pct_ret'] = data['Adj Close'].pct_change()
data.tail()

,Open,High,Low,Adj Close,pct_ret
Date,,,,,
2018-12-21,246.740005,249.710007,239.979996,229.649750,-0.020490
2018-12-24,239.039993,240.839996,234.270004,223.581726,-0.026423
2018-12-26,235.970001,246.179993,233.759995,234.878159,0.050525
2018-12-27,242.570007,248.289993,238.960007,236.681427,0.007677
2018-12-28,249.580002,251.399994,246.449997,236.376099,-0.001290


# **Markov Chains**

**A Markov chain is a type of stochastic process.**

**A Markov chain is a collection of random variables ($X_t$) where the future states ($j$) only depend on the current state ($i$). Markov chains can be either discrete or continuous.**

**For a Markov chain transition matrix (denoted as $P$):**

 - **Each row must add to one, where: $\sum\limits_{j}P_{ij} = 1$.**
 - **The probabilities must be non-negative where: $P_{ij} \geq 0 \ \ \forall \ \ i,j$**

In [78]:
# compute the states
data['state'] = data['pct_ret'].apply(lambda x: 'Up' if (x > 0.001)
                                     else ('Down' if (x < -0.001)
                                           else 'Flat' ))
data.tail()


,Open,High,Low,Adj Close,pct_ret,state
Date,,,,,,
2018-12-21,246.740005,249.710007,239.979996,229.649750,-0.020490,Down
2018-12-24,239.039993,240.839996,234.270004,223.581726,-0.026423,Down
2018-12-26,235.970001,246.179993,233.759995,234.878159,0.050525,Up
2018-12-27,242.570007,248.289993,238.960007,236.681427,0.007677,Up
2018-12-28,249.580002,251.399994,246.449997,236.376099,-0.001290,Down


In [79]:
# compute the prior state
data['priorstate'] = data['state'].shift(1)
data.tail()

,Open,High,Low,Adj Close,pct_ret,state,priorstate
Date,,,,,,,
2018-12-21,246.740005,249.710007,239.979996,229.649750,-0.020490,Down,Down
2018-12-24,239.039993,240.839996,234.270004,223.581726,-0.026423,Down,Down
2018-12-26,235.970001,246.179993,233.759995,234.878159,0.050525,Up,Down
2018-12-27,242.570007,248.289993,238.960007,236.681427,0.007677,Up,Up
2018-12-28,249.580002,251.399994,246.449997,236.376099,-0.001290,Down,Up


In [80]:
# Frequency distributions
states = data[['priorstate', 'state']].dropna()
states_mat = states.groupby(['priorstate', 'state']).size().unstack()
states_mat

state,Down,Flat,Up
priorstate,,,
Down,46,6,50
Flat,18,3,11
Up,39,22,54


In [81]:
# Initial transition matrix
transition_matrix = states_mat.apply(lambda x: x / float(x.sum()), axis=1)
transition_matrix

state,Down,Flat,Up
priorstate,,,
Down,0.45098,0.058824,0.490196
Flat,0.56250,0.093750,0.343750
Up,0.33913,0.191304,0.469565


In [82]:
# find the transition matrix at t1
t0 = transition_matrix.copy()
t1 = round(t0.dot(t0),4)
t1

state,Down,Flat,Up
priorstate,,,
Down,0.4027,0.1258,0.4715
Flat,0.4230,0.1076,0.4694
Up,0.4198,0.1277,0.4525


In [83]:
# find the transition matrix at t2
t2 = round(t0.dot(t1),4)
t2

state,Down,Flat,Up
priorstate,,,
Down,0.4123,0.1257,0.4621
Flat,0.4105,0.1247,0.4648
Up,0.4146,0.1232,0.4622


In [84]:
# find the transition matrix at t3
t3 = t0.dot(t2)
t3

state,Down,Flat,Up
priorstate,,,
Down,0.413322,0.124416,0.462308
Flat,0.412922,0.124747,0.462387
Up,0.413036,0.124335,0.462663


In [85]:
# find the equilibrium matrix
i = 1
a = t0.copy()
b = t0.dot(t0)
while(not(a.equals(b))):
    print("Iteration number: " + str(i))
    i += 1
    a = b.copy()
    b = b.dot(t0)
# Raises the matrix to the power of n-day
    mat=  pd.DataFrame(np.linalg.matrix_power(t0, 4))
mat

Iteration number: 1
Iteration number: 2
Iteration number: 3
Iteration number: 4
Iteration number: 5
Iteration number: 6
Iteration number: 7
Iteration number: 8
Iteration number: 9
Iteration number: 10
Iteration number: 11
Iteration number: 12
Iteration number: 13
Iteration number: 14
Iteration number: 15
Iteration number: 16
Iteration number: 17
Iteration number: 18
Iteration number: 19
Iteration number: 20
Iteration number: 21


,0,1,2
0,0.413316,0.124425,0.462259
1,0.412912,0.124752,0.462336
2,0.413031,0.124355,0.462615
